<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/ANN_parte_002_B_hidrofoil_ajuste_ann_SET_12_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# este comando clona o diretorio github ucfilho/ANN no colaboratory research
!git clone https://github.com/ucfilho/ANN/ #clonar do Github
%cd ANN
#!ls
print("Hello world"+"\n"+"I'm evolving!!!"+"\n")

%matplotlib inline

fatal: destination path 'ANN' already exists and is not an empty directory.
/content/ANN
Hello world
I'm evolving!!!



In [0]:
# sigmoid function
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

def tasig(x,deriv=False):
    if(deriv==True):
        return 1-x*x
    return 2/(1+np.exp(-2*x))-1

# purelin function
def purelin(x,deriv=False):
    if(deriv==True):
        return 1
    return x

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [5]:
# leitura dos dados
DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv")
# separa as colunas de interesse
Index=[2,3,4,5,6,7]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()
print(Dat_Select)

    Hydrofoil    Froude  Resistence     Sinkage      Trim        Power
0           1  0.000000    0.000000    0.000000  0.000000     0.000000
1           1  0.103733    2.549729   -0.689028  1.353022     0.950991
2           1  0.124480    3.628461   -0.835413  1.346323     1.624000
3           1  0.165973    6.374322   -1.864337  1.361538     3.803965
4           1  0.186719    8.217973   -2.301539  1.378858     5.517212
5           1  0.207466   10.365629   -1.506604  1.369050     7.732290
6           1  0.414932   66.979419   -6.500115  2.054651    99.927230
7           1  0.622398  129.545847    9.513175  4.409404   289.906013
8           1  0.829864  153.768272   25.026706  4.280094   458.816682
9           1  1.037330  184.757286   39.739036  4.495847   689.102861
10          1  1.244797  224.572285   49.763887  4.369803  1005.124556
11          1  1.452263  250.952173   57.493840  4.472855  1310.392734
12          1  1.659729  262.818220   73.815676  5.006382  1568.403965
13    

In [0]:
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [7]:
# obtem as variaveis independentes (treino e teste)
# separa os dados em treino e teste
train=DAT.sample(frac=0.8,random_state=None) # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index)
x_train=train.iloc[:,[0,1]]
x_test=test.iloc[:,[0,1]]
X_OLD=DAT_OLD.iloc[:,[0,1]]
print(x_train)

    Hydrofoil    Froude
56   1.000000  0.728608
35   0.333333 -0.308557
7   -1.000000 -0.481417
16  -1.000000  1.000000
31   0.333333 -1.000000
43   0.333333  0.797753
25  -0.333333  0.382887
9   -1.000000 -0.135696
48   1.000000 -0.481417
29  -0.333333  0.866897
32   0.333333 -0.827139
24  -0.333333  0.210026
10  -1.000000  0.037165
27  -0.333333  0.688850
14  -1.000000  0.688850
46   1.000000 -0.827139
28  -0.333333  0.728608
34   0.333333 -0.481417
6   -1.000000 -0.654278
21  -0.333333 -0.308557
45   1.000000 -1.000000
26  -0.333333  0.555748
41   0.333333  0.688850
23  -0.333333  0.037165
1   -1.000000 -0.913570
47   1.000000 -0.654278
53   1.000000  0.382887
4   -1.000000 -0.844425
11  -1.000000  0.210026
40   0.333333  0.555748
52   1.000000  0.210026
55   1.000000  0.688850
50   1.000000 -0.135696
37   0.333333  0.037165
49   1.000000 -0.308557
0   -1.000000 -1.000000
5   -1.000000 -0.827139
18  -0.333333 -0.827139
54   1.000000  0.555748
15  -1.000000  0.728608
38   0.333333  0

In [8]:
# obtem as variaveis dependentes (treino e teste)
#Index=[2,3,4,5]
Index=[2]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
print(y_train)

    Resistence
56    0.458333
35   -0.179688
7    -0.311979
16    0.993750
31   -1.000000
43    0.609375
25    0.281250
9    -0.018750
48   -0.278646
29    0.901042
32   -0.942188
24    0.130208
10    0.192708
27    0.651042
14    0.552083
46   -0.940104
28    0.697917
34   -0.265625
6    -0.644271
21   -0.179688
45   -1.000000
26    0.442708
41    0.520833
23    0.054688
1    -0.986458
47   -0.619792
53    0.118750
4    -0.956354
11    0.332812
40    0.341146
52   -0.015625
55    0.437500
50   -0.122917
37   -0.019792
49   -0.210937
0    -1.000000
5    -0.944948
18   -0.940885
54    0.244792
15    0.626042
38    0.041667
30    1.000000
2    -0.980729
8    -0.183333
36   -0.078125
19   -0.630208
58    0.520833


In [0]:
#print(len(y_old))
#print(Y_test['CR'])

In [0]:
# input dataset
#Neuron_input=3
Neuron_input=2
Neuron_hidden=5
Neuron_output=1
Learning_rate=0.01
omega=0.9
Epoch=500000

In [0]:

#y_old=DAT['TC1']
#DAT=normatize(DAT)
Y_train=y_train
X_train=x_train
#print(X.head())

In [12]:
X_NOME=['Hydrofoil','Froude']
Y_NOME=['Resistence']   
#X=DAT.loc[DAT['Speed']==DAT['Speed'],['Speed','Feed','Hard']].values
#y=DAT.loc[DAT['Speed']==DAT['Speed'],['TC1']].values
X_train=X_train.loc[X_train['Hydrofoil']==X_train['Hydrofoil'],X_NOME].values
y=Y_train.loc[Y_train['Resistence']==Y_train['Resistence'],['Resistence']].values
#X=XX
#np.random.seed(1)

# randomly initialize our weights with mean 0
syn0 = 2*np.random.random((Neuron_input,Neuron_hidden)) - 1
syn1 = 2*np.random.random((Neuron_hidden,Neuron_output)) - 1


for j in range(Epoch):

    # Feed forward through layers 0, 1, and 2
    l0 = X_train
    l1 = sigmoid(np.dot(l0,syn0))
    l2 = purelin(np.dot(l1,syn1))

    # how much did we miss the target value?
    l2_error = y - l2

    if (j% 2000) == 0:
        print("Epoch %d"%(j) +" error:" + str(np.mean(np.abs(l2_error))))

    # in what direction is the target value?
    # were we really sure? if so, don't change too much.
    l2_delta = l2_error*purelin(l2,deriv=True)
    l1_error = l2_delta.dot(syn1.T)
    l1_delta = l1_error * sigmoid(l1,deriv=True)

    syn1 += l1.T.dot(l2_delta)*Learning_rate
    syn0 += l0.T.dot(l1_delta)*Learning_rate



Epoch 0 error:0.7237360814673475
Epoch 2000 error:0.10827857572381602
Epoch 4000 error:0.10688731047930482
Epoch 6000 error:0.10626974111426174
Epoch 8000 error:0.10589915998342272
Epoch 10000 error:0.1056451812776629
Epoch 12000 error:0.10545483031327912
Epoch 14000 error:0.1053034263512318
Epoch 16000 error:0.10517821574976961
Epoch 18000 error:0.10507185563159163
Epoch 20000 error:0.10497970788771066
Epoch 22000 error:0.1048986366455432
Epoch 24000 error:0.10482641819449594
Epoch 26000 error:0.10476142070269955
Epoch 28000 error:0.1047024152417142
Epoch 30000 error:0.10464845696927216
Epoch 32000 error:0.10459880674681696
Epoch 34000 error:0.1045528774927504
Epoch 36000 error:0.10451039165885256
Epoch 38000 error:0.10447905478710931
Epoch 40000 error:0.10444988139861991
Epoch 42000 error:0.10442262123782223
Epoch 44000 error:0.10439706543886702
Epoch 46000 error:0.10437303799505172
Epoch 48000 error:0.1043503892235437
Epoch 50000 error:0.10432899071875723
Epoch 52000 error:0.1043087

In [19]:
#'''
#DAT=pd.read_csv(Arquivo,sep='\t')
y_obs=y_train
y_calc=np.reshape(l2,len(l2))
y_calc=pd.DataFrame({'y_calc':y_calc})
#y_old=pd.DataFrame({'y_old':y_old})
Y_OLD=pd.DataFrame(Y_OLD)
Y_OLD.columns = ["y_old"]
y_calc=Original(y_calc,Y_OLD)


plt.scatter(y_obs,y_calc)
plt.xlabel("True CR")
plt.ylabel("Estimated CR")
plt.xlim((0,1))
plt.ylim((0,1))
plt.show()
#'''

KeyError: ignored

In [20]:
print(Y_OLD)
print(y_calc)

         y_old
0     0.000000
1     2.549729
2     3.628461
3     6.374322
4     8.217973
5    10.365629
6    66.979419
7   129.545847
8   153.768272
9   184.757286
10  224.572285
11  250.952173
12  262.818220
13  284.392850
14  292.238170
15  306.163613
16  375.398562
17    0.000000
18   11.130548
19   69.627215
20  136.802768
21  154.454737
22  177.500365
23  198.584663
24  212.804305
25  241.243590
26  271.644205
27  310.870805
28  319.696790
29  357.942725
30  376.575360
31    0.000000
32   10.885381
33   70.607880
34  138.273765
35  154.454737
36  173.577705
37  184.561153
38  196.133000
39  223.591620
40  252.521238
41  286.354180
42  296.160830
43  303.025485
44  333.426100
45    0.000000
46   11.277648
47   71.588545
48  135.822102
49  148.570748
50  165.143986
51  174.558370
52  185.345685
53  210.646842
54  234.378935
55  270.663540
56  274.586200
57  291.257505
58  286.354180
      y_calc
0   0.561331
1  -0.366171
2  -0.433538
3   0.888619
4  -1.006324
5   0.684245
6   0.332

In [0]:
#print(y_old)
X_test=X_test.loc[X_test['VELOC X']==X_test['VELOC X'],X_NOME].values
y_calc=np.reshape(l2,len(l2))
y_calc=pd.DataFrame({'y_calc':y_calc})

#print(y_calc)
#print(y_old)
#print(l2)
#print(np.reshape(l2,len(l2)))

In [0]:
#print(l0)

In [0]:

#print(X_test)
l0 = X_test
l1 = sigmoid(np.dot(l0,syn0))
l2 = purelin(np.dot(l1,syn1))

In [0]:
y_calc_test=np.reshape(l2,len(l2))
y_calc_test=pd.DataFrame({'y_calc_test':y_calc_test})
y_calc_test=original(y_calc_test,y_total)
#y_old=pd.DataFrame(y_old)
#y_old.columns = ["y_old"]

In [0]:
print(y_calc_test)

In [0]:
plt.scatter(y_obs,y_calc)
plt.scatter(Y_test['CR'],y_calc_test,color='r')
plt.xlabel("True CR")
plt.ylabel("Estimated CR")
plt.xlim((-2,3))
plt.ylim((-2,3))
plt.show()

In [0]:
#print(l1)

In [0]:
#print(l2)